In [1]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.3.19 \
     langchain-groq==0.2.4 \
     langchain-chroma==0.2.2 \
     langchain-community==0.3.18 \
     langchain-huggingface==0.1.2 \
     einops==0.8.1

In [2]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import WebBaseLoader
from chromadb.config import Settings
import chromadb

In [3]:
import os
import getpass

In [4]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


In [5]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


Step 1 : Load and preprocess data

In [6]:
def load_and_process_data(url):
    # Load data from web
    loader = WebBaseLoader(url)
    data = loader.load()

    # Split text into chunks (Experiment with Chunk Size and Chunk Overlap to get optimal chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(data)

    return chunks

Step 2: Create multiple vector stores code

In [7]:
def create_vector_stores(chunks_list):
    embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs={'trust_remote_code': True})

    # 1. Define settings (this is correct)
    client_settings = Settings(anonymized_telemetry=False)

    # 2. Create the client ONCE, outside the loop
    # We use .Client() for an in-memory "ephemeral" client
    client = chromadb.Client(client_settings)

    vector_stores = []

    # 3. Loop and create collections
    for i, chunks in enumerate(chunks_list):

        # Create a unique name
        collection_name = f"source_collection_{i}"

        # 4. Pass the EXISTING client and a unique name.
        #    DO NOT pass client_settings here again.
        vector_store = Chroma.from_documents(
            chunks,
            embeddings,
            client=client,  # <-- Pass the client you already made
            collection_name=collection_name  # <-- Pass the unique name
        )
        vector_stores.append(vector_store)

    return vector_stores

Step 3: Mult-index-RAG related code

1. **Retrieval from Multiple Indices**: We retrieve relevant documents from each vector store.
2. **Context Combination:** We combine the retrieved documents from all sources into a single context.
3. **Response Generation:** Using the combined context, we generate a final response to the original query.
4. **Source Usage Explanation:** We generate an explanation of how information from different sources was used to answer the question.

In [8]:
def multi_index_rag(query, vector_stores, llm):
    # Retrieve documents from each vector store
    all_docs = []
    for i, vector_store in enumerate(vector_stores):
        docs = vector_store.similarity_search(query, k=3)  # Increased k to 3
        all_docs.extend([f"Source {i+1}: " + doc.page_content for doc in docs])

    # Combine retrieved documents
    context = "\n\n".join(all_docs)

    # Generate response using combined context
    response_prompt = ChatPromptTemplate.from_template(
        "You are an AI assistant tasked with answering questions based on the provided context. "
        "The context contains information from multiple sources related to AI, machine learning, and NLP. "
        "Please analyze the context carefully and provide a comprehensive answer to the question. "
        "If the context doesn't contain enough information, use your general knowledge to supplement the answer, "
        "but prioritize information from the context when available.\n\n"
        "Context:\n{context}\n\n"
        "Question: {query}\n"
        "Answer:"
    )
    response_chain = response_prompt | llm
    try:
        response = response_chain.invoke({"context": context, "query": query})
        final_answer = response.content
    except Exception as e:
        print(f"Error generating response: {e}")
        final_answer = "I apologize, but I encountered an error while generating the response."

    # Generate explanation of source usage
    explanation_prompt = ChatPromptTemplate.from_template(
        "Based on the answer you provided, explain how information from different sources was used to answer the question. "
        "If general knowledge was used to supplement the answer, mention that as well.\n\n"
        "Context: {context}\n"
        "Question: {query}\n"
        "Answer: {answer}\n"
        "Explanation:"
    )
    explanation_chain = explanation_prompt | llm
    try:
        explanation = explanation_chain.invoke({"context": context, "query": query, "answer": final_answer})
        source_explanation = explanation.content
    except Exception as e:
        print(f"Error generating explanation: {e}")
        source_explanation = "Unable to generate explanation due to an error."

    return {
        "final_answer": final_answer,
        "source_explanation": source_explanation,
        "retrieved_context": context
    }

Step 4: Create chunk of web data to Chroma Vector Store

In [10]:
!pip install --upgrade --force-reinstall numpy pandas scipy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 135.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 163.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 35.5 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfu

In [11]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.5
)

# Load and process data
urls = [
        "https://en.wikipedia.org/wiki/Artificial_intelligence",
        "https://en.wikipedia.org/wiki/Machine_learning",
        "https://en.wikipedia.org/wiki/Natural_language_processing"
      ]

print("Loading and processing data...")
chunks_list = [load_and_process_data(url) for url in urls]

# Create multiple vector stores
print("Creating vector stores...")
vector_stores = create_vector_stores(chunks_list)
print("Vector stores created successfully.")

Loading and processing data...
Creating vector stores...


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Vector stores created successfully.


Step 5: Run Multi-index RAG

This implementation shows the key parts of Multi-index RAG:

1. Use of multiple vector stores representing different sources or types of information
2. Retrieval and combination of information from multiple sources
Generation of a comprehensive response using the combined information
3. Explanation of how different sources contributed to the answer

In [12]:
# Example query
query = "How do AI, machine learning, and NLP relate to each other in modern applications?"

# Run Multi-index RAG
result = multi_index_rag(query, vector_stores, llm)

print("Query:", query)
print("\nFinal Answer:")
print(result["final_answer"])
print("\nSource Usage Explanation:")
print(result["source_explanation"])

Query: How do AI, machine learning, and NLP relate to each other in modern applications?

Final Answer:
Based on the provided context, AI, machine learning, and NLP are closely related concepts that have evolved together in modern applications. Here's a comprehensive overview of their relationship:

1. **AI and Machine Learning**: Machine learning is a subfield of AI that focuses on developing programs that can improve their performance on a given task automatically. As mentioned in the context, machine learning has been a part of AI from the beginning and has been a driving force behind the success of AI research in recent years.

2. **Machine Learning and NLP**: Machine learning is a key enabler of NLP, which involves enabling computers to process, understand, and generate human language. As mentioned in Source 3, NLP-powered Document AI enables non-technical teams to quickly access information hidden in documents, and machine learning approaches, including statistical and neural net